Backup and run loacally at:
https://model.earth/RealityStream/models/location-forest

Load Libraries and data

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Get the raw file contents directly without any HTML wrapping or formatting
df = pd.read_csv("https://raw.githubusercontent.com/ModelEarth/RealityStream/main/input/bees/locations/bees-locations.csv")
# df = pd.read_csv("../../input/bees/locations/bees-locations.csv")

df = df[['Year', 'State', 'State ANSI', 'Ag District', 'Ag District Code', 'County',
       'County ANSI','Value']]
df = df.pivot(index=['State', 'State ANSI', 'Ag District', 'Ag District Code', 'County',
       'County ANSI',], columns='Year', values='Value').reset_index()
df = df.rename_axis(None, axis=1)
df = df.replace(" (D)", np.NaN)


Preprocess the data  
TO DO: Modify to work with all input data, regardless of year

In [ ]:
# remove ',' in number
df[2002] = df[2002].str.replace(',', '')
df[2007] = df[2007].str.replace(',', '')
df[2012] = df[2012].str.replace(',', '')
df[2017] = df[2017].str.replace(',', '')
df[2022] = df[2022].str.replace(',', '')
# convert columns to int date type
df[2002] = df[2002].fillna(0).astype(str).astype(int)
df[2007] = df[2007].fillna(0).astype(str).astype(int)
df[2012] = df[2012].fillna(0).astype(str).astype(int)
df[2017] = df[2017].fillna(0).astype(str).astype(int)
df[2022] = df[2022].fillna(0).astype(str).astype(int)
# replace 0 back to nan
df[2002] = df[2002].replace(0, np.NaN)
df[2007] = df[2007].replace(0, np.NaN)
df[2012] = df[2012].replace(0, np.NaN)
df[2017] = df[2017].replace(0, np.NaN)
df[2022] = df[2022].replace(0, np.NaN)
# replace outliers to nan
df[2002] = df[2002].mask(df[2012].sub(df[2012].mean()).div(df[2012].std()).abs().gt(2))
df[2007] = df[2007].mask(df[2012].sub(df[2012].mean()).div(df[2012].std()).abs().gt(2))
df[2012] = df[2012].mask(df[2012].sub(df[2012].mean()).div(df[2012].std()).abs().gt(2))
df[2017] = df[2017].mask(df[2017].sub(df[2017].mean()).div(df[2017].std()).abs().gt(2))
df[2022] = df[2022].mask(df[2022].sub(df[2022].mean()).div(df[2022].std()).abs().gt(2))
# impute nan with mean
df[2002].fillna(value=round(df[2012].mean()), inplace=True)
df[2007].fillna(value=round(df[2012].mean()), inplace=True)
df[2012].fillna(value=round(df[2012].mean()), inplace=True)
df[2017].fillna(value=round(df[2017].mean()), inplace=True)
df[2022].fillna(value=round(df[2022].mean()), inplace=True)
df['2007_increase'] = df.apply(lambda x: 0 if (x[2007] - x[2002]) < 0 else 1, axis=1)
df['2012_increase'] = df.apply(lambda x: 0 if (x[2012] - x[2007]) < 0 else 1, axis=1)
df['2017_increase'] = df.apply(lambda x: 0 if (x[2017] - x[2012]) < 0 else 1, axis=1)
df['2022_increase'] = df.apply(lambda x: 0 if (x[2022] - x[2017]) < 0 else 1, axis=1)
df['County'] = df['County'].apply(lambda x: x.title()+' County')

df_fips is the dataset for national bee population.

In [ ]:
states = ["AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "IA",
          "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", "ME", "MI", "MN", "MO",
          "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM", "NV", "NY", "OH", "OK",
          "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA", "WI",
          "WV", "WY"]
year = 2021
naics_value = 4
df_full = pd.DataFrame()
for state in states:
  url = f"https://raw.githubusercontent.com/ModelEarth/community-timelines/main/training/naics{naics_value}/US/counties/{year}/US-{state}-training-naics{naics_value}-counties-{year}.csv"
  df_ind = pd.read_csv(url)
  df_full = pd.concat([df_full, df_ind], axis=0, join='outer')

df_fips = df_full[['Name','Fips']].merge(df, how='outer', left_on='Name', right_on='County')
df_fips.to_csv('../../input/targets/bees-targets.csv')